In [1]:
%matplotlib notebook
import numpy as np
import scipy.linalg as la
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

import scipy.sparse as sp
from scipy.sparse.linalg import spsolve, lsqr

from numpy.linalg import cond, norm, inv, lstsq

from scipy.optimize import minimize_scalar

from poly_basis import *
from spherepts import *
from rbf import *

from laplacebeltrami import *

from math import factorial as fac

In [10]:
n = 400
eps = None
k = 50
#rbf_obj = rbf_dict['multiquadric']
rbf_obj = rbf_dict['r^9']
# Choose solution and forcing fuction
solution_index = 20

nodes = gen_spiral_nodes(n)
normals = nodes

rbf = rbf_obj['rbf']
zeta  = rbf_obj['zeta']
d2rbf = rbf_obj['d2rbf']
Lrbf = lambda r,eps: 1*zeta(r,eps) + d2rbf(r,eps)

sol_deg = sphere_harm_degs[solution_index]
exact = lambda x: sphere_harm[solution_index](*x)*-sol_deg*(sol_deg+1)
forcing = lambda x: sphere_harm[solution_index](*x)
print('Harmonic degree: %d' % sol_deg)

Harmonic degree: 4


# Shankar-Wright Method

In [11]:
C = SWM(nodes, normals, rbf_obj, eps, k, poly_deg=2, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

3.4765217129917323


In [12]:
C = SWM(nodes, normals, rbf_obj, eps, k, poly_deg=1, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

0.001904940403098113


In [13]:
C = SWM(nodes, normals, rbf_obj, eps, k, poly_deg=4, poly_type='p')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

0.0005002980927061657


In [14]:
C = SWM(nodes, normals, rbf_obj, eps, k, poly_deg=2, poly_type='s')
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

3.476521860070867


# Tangent Plane Method

In [7]:
C = TPM(nodes, normals, rbf_obj, eps, k, poly_deg=2)
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

0.00048721016990151497


In [8]:
C = TPM(nodes, normals, rbf_obj, eps, k, poly_deg=4)
fs = np.array([forcing(node) for node in nodes])
ds = C @ fs
es = np.array([exact(node) for node in nodes])
print(la.norm(ds - es)/la.norm(es))

0.00024036219976002443


In [9]:
|

SyntaxError: invalid syntax (<ipython-input-9-4b37ef281455>, line 1)